In [3]:
from sqlalchemy import create_engine
import os, json
import pandas as pd
import pymysql
pymysql.install_as_MySQLdb()

In [13]:
engine = create_engine(os.environ['SQLALCHEMY_DATABASE_URI']+'?charset=utf8')

In [14]:
# Read table
notebooks = pd.read_sql_table('notebooks', engine).sort_values('id').drop('notebook_url', axis=1)
notebooks.head()

# Reset id
notebooks['id'] = [x+1 for x in range(len(notebooks.index))]

# Add dataset
notebooks['dataset'] = [rowData['gse'] if rowData['gse'] else json.loads(rowData['notebook_configuration'])['data']['parameters']['uid'] for index, rowData in notebooks.iterrows()]

# Drop gse
notebooks.drop('gse', axis=1, inplace=True)

# Title
notebooks['notebook_title'] = [json.loads(x)['notebook']['title'] if json.loads(x)['notebook']['title'] else '' for x in notebooks['notebook_configuration']]

# Add time
notebooks['time'] = None
notebooks.head()

,id,notebook_uid,version,date,notebook_configuration,dataset,notebook_title,time
0,1,bfxNc9ga5,v0.3,2018-03-05 11:04:54,"{""notebook"": {""title"": ""GSE88741 Analysis Note...",GSE88741,GSE88741 Analysis Notebook,None
1,2,L7PlI61ii,v0.3,2018-03-05 11:05:42,"{""notebook"": {""title"": ""GSE88741 Analysis Note...",GSE88741,GSE88741 Analysis Notebook,None
2,3,KOk4C6XeW,v0.3,2018-03-05 11:06:27,"{""notebook"": {""title"": ""GSE88741 Analysis Note...",GSE88741,GSE88741 Analysis Notebook,None
3,4,u0ihfVbuq,v0.3,2018-03-05 11:08:55,"{""notebook"": {""title"": ""GSE88741 Analysis Note...",GSE88741,GSE88741 Analysis Notebook,None
4,5,IxTXstW4V,v0.3,2018-03-05 11:16:37,"{""notebook"": {""title"": ""GSE88741 Analysis Note...",GSE88741,GSE88741 Analysis Notebook,None


In [15]:
# Get tool ids
tool_dict = pd.read_sql_table('tool', engine).set_index('tool_string')['id'].to_dict()

# Prepare dataframe
notebook_tool_dataframe = pd.concat([pd.DataFrame({'notebook_fk': rowData['id'], 'tool_fk': [tool_dict.get(x['tool_string']) for x in json.loads(rowData['notebook_configuration'])['tools']]}) for index, rowData in notebooks.iterrows()]).dropna()
notebook_tool_dataframe.head()

,notebook_fk,tool_fk
0,1,1.0
0,2,1.0
0,3,1.0
0,4,1.0
0,5,1.0


In [16]:
# Truncate
engine.execute('SET FOREIGN_KEY_CHECKS=0;')
engine.execute('TRUNCATE TABLE notebook;')
engine.execute('TRUNCATE TABLE notebook_tool;')
engine.execute('SET FOREIGN_KEY_CHECKS=1;')

# Upload
notebooks.to_sql('notebook', engine, if_exists='append', index=False)
notebook_tool_dataframe.to_sql('notebook_tool', engine, if_exists='append', index=False)

In [46]:
gpl = 'GPL17021'

In [45]:
import xmltodict
import requests

In [71]:
import urllib.request
gpl_organism = {}
for gpl in pd.read_sql_table('platform_new', engine)['platform_accession'].unique():
    with urllib.request.urlopen('https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=gds&term={}'.format(gpl)) as search_response:
        gpl_id = xmltodict.parse(search_response.read())['eSearchResult']['IdList']['Id'][0]
        with urllib.request.urlopen('https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=gds&id={}'.format(gpl_id)) as summary_response:
            organism = xmltodict.parse(summary_response.read())['eSummaryResult']['DocSum']['Item'][6]['#text'].replace('Mus musculus', 'Mouse').replace('Homo sapiens', 'Human')
            gpl_organism[gpl] = organism

In [72]:
gpl_organism

{'GPL11154': 'Human',
 'GPL13112': 'Mouse',
 'GPL16791': 'Human',
 'GPL17021': 'Mouse',
 'GPL18573': 'Human',
 'GPL19057': 'Mouse; Human'}

## Signature

In [42]:
sample_dataframe = pd.read_sql_table('sample_new', engine)
sample_dataframe.head()

,id,sample_accession,sample_title,dataset_fk,platform_fk
0,1,GSM2668542,61_Old_CS,1,1
1,2,GSM2668543,63_Old_CS,1,1
2,3,GSM2668544,65_Old_CS,1,1
3,4,GSM2668545,67_Old_Naive,1,1
4,5,GSM2668546,69_Old_Naive,1,1


In [24]:
notebook = pd.read_sql_table('notebook', engine)
notebook['notebook_configuration'] = [json.loads(x) for x in notebook['notebook_configuration']]
notebook.head()

,id,notebook_uid,dataset,notebook_title,version,notebook_configuration,time,date
0,1,bfxNc9ga5,GSE88741,GSE88741 Analysis Notebook,v0.3,{'notebook': {'title': 'GSE88741 Analysis Note...,NaN,2018-03-05 11:04:54
1,2,L7PlI61ii,GSE88741,GSE88741 Analysis Notebook,v0.3,{'notebook': {'title': 'GSE88741 Analysis Note...,NaN,2018-03-05 11:05:42
2,3,KOk4C6XeW,GSE88741,GSE88741 Analysis Notebook,v0.3,{'notebook': {'title': 'GSE88741 Analysis Note...,NaN,2018-03-05 11:06:27
3,4,u0ihfVbuq,GSE88741,GSE88741 Analysis Notebook,v0.3,{'notebook': {'title': 'GSE88741 Analysis Note...,NaN,2018-03-05 11:08:55
4,5,IxTXstW4V,GSE88741,GSE88741 Analysis Notebook,v0.3,{'notebook': {'title': 'GSE88741 Analysis Note...,NaN,2018-03-05 11:16:37


In [30]:
for group in ['A', 'B']:
    notebook[group] = [x['signature'][group]['samples'] if len(x['signature']) else None for x in notebook['notebook_configuration']]

In [35]:
signature = notebook[['notebook_uid', 'A', 'B']].rename(columns={'A': 'control', 'B': 'perturbation'}).dropna()
signature.head()

,notebook_uid,control,perturbation
4,IxTXstW4V,"[GSM2344965, GSM2344966, GSM2344967]","[GSM2344971, GSM2344972, GSM2344973]"
5,eB0JLr3d3,"[GSM2344965, GSM2344966, GSM2344967]","[GSM2344971, GSM2344972, GSM2344973]"
6,nChgjTx4f,"[GSM2344965, GSM2344966, GSM2344967]","[GSM2344971, GSM2344972, GSM2344973, GSM234496..."
7,pEBOp87zR,"[GSM2686061, GSM2686062, GSM2686063]","[GSM2686058, GSM2686059, GSM2686060]"
8,z9mIfboIP,"[GSM2682871, GSM2682872, GSM2682873, GSM2682879]","[GSM2682874, GSM2682875, GSM2682877]"


In [38]:
results = []
for index, rowData in signature.iterrows():
    for col in ['control', 'perturbation']:
        for sample in rowData[col]:
            results.append({'notebook_uid': rowData['notebook_uid'], 'sample_accession': sample, 'control': col == 'control'})
            
result_dataframe = pd.DataFrame(results)
result_dataframe.head()

,control,notebook_uid,sample_accession
0,True,IxTXstW4V,GSM2344965
1,True,IxTXstW4V,GSM2344966
2,True,IxTXstW4V,GSM2344967
3,False,IxTXstW4V,GSM2344971
4,False,IxTXstW4V,GSM2344972


In [44]:
result_dataframe_fk = result_dataframe.merge(notebook.rename(columns={'id': 'notebook_fk'}), on='notebook_uid')[['notebook_fk', 'sample_accession', 'control']].merge(sample_dataframe.rename(columns={'id': 'sample_fk'}), on='sample_accession')[['notebook_fk', 'sample_fk', 'control']]
result_dataframe_fk.head()

,notebook_fk,sample_fk,control
0,5,184695,True
1,6,184695,True
2,7,184695,True
3,42,184695,True
4,47,184695,True


In [45]:
result_dataframe_fk.shape

(3016, 3)

In [46]:
result_dataframe_fk.groupby('control').size()

control
False    1662
True     1354
dtype: int64